# Optimization example

We want to maximize the energy production benefits of running 2 generators. The benefit of producing with generator 1 and 2 are 3&euro;/MWh and 5&euro;/MWh, respectively. The maximum capacity of generator 1 and 2 are 4MW and 6MW, respectively. Both generators share the same water cooling system. Generator 1 needs 3 units of water per MW, while generator 2 needs 2 units of water per MW. The maximum units of water available is 18.

This optimization model is formulated as follows:

$$
\begin{align}
\underset{x_1,x_2}{\max} \quad & 3x_1+5x_2 \\
\text{s.t.} \quad & 0 \leq x_1 \leq 4\\
& 0 \leq x_2 \leq 6\\
& 3x_1 + 2x_2 \leq 18
\end{align}
$$


## REQUIREMENTS

In [19]:
# Gurobipy
!pip install -q gurobipy
import gurobipy as gp
from gurobipy import GRB
# Pulp
!pip install -q pulp
from pulp import LpMaximize, LpProblem, LpVariable, lpSum
# Scipy
!pip install -q scipy
from scipy.optimize import linprog
# Pyomo + glkp
!apt-get install -y -qq glpk-utils
!pip install -q pyomo
import pyomo.environ as pe
glpk = pe.SolverFactory('glpk', executable='/usr/bin/glpsol')

AttributeError: '_PyDrive2ImportHook' object has no attribute 'find_spec'

## PULP ([link](https://coin-or.github.io/pulp/))

PuLP is a user-friendly library for linear programming. It is easy to learn and integrates well with other Python libraries.

In [ ]:
# Model
model = LpProblem(name='small-problem', sense=LpMaximize)
# Variables
x1 = LpVariable(name='x1', lowBound=0, upBound=4)
x2 = LpVariable(name='x2', lowBound=0, upBound=6)
# Objective function
model += lpSum([3 * x1, 5 * x2])
# Constraints
model += (3 * x1 + 2 * x2 <= 18, 'water_constraint')
# Solve the problem
model.solve()
# Print results
print('x1 =', x1.varValue)
print('x2 =', x2.varValue)
print('Optimal value =', model.objective.value())
# Explanation: This section demonstrates how to solve the optimization problem using PuLP.

## SCIPY ([link](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html))

Scipy is a general-purpose library for scientific computing. It includes a simple linear programming solver that is easy to use for small to medium-sized problems.

In [ ]:
# Coefficients of the objective function
c = [-3, -5]  # Note: linprog does minimization by default
# Coefficients of the inequality constraints
A = [[3, 2]]
# Right-hand side of the inequality constraints
b = [18]
# Bounds for the variables
x0_bounds = (0, 4)
x1_bounds = (0, 6)
# Solve the problem
res = linprog(c, A_ub=A, b_ub=b, bounds=[x0_bounds, x1_bounds], method='highs')
# Print results
print('x1 =', res.x[0])
print('x2 =', res.x[1])
print('Optimal value =', -res.fun)  # Negate because we minimized
# Explanation: This section demonstrates how to solve the optimization problem using Scipy.

## PYOMO + GLPK ([link](https://pyomo.readthedocs.io/en/stable/))

Pyomo is a versatile library that can handle a wide range of optimization problems. It is particularly useful for complex models and supports various solvers.

In [ ]:
# Model
m = pe.ConcreteModel()
# Variables
m.x1 = pe.Var(within=pe.NonNegativeReals,bounds=(0,4))
m.x2 = pe.Var(within=pe.NonNegativeReals,bounds=(0,6))
# Objective function
m.obj = pe.Objective(expr = 3*m.x1 + 5*m.x2,sense=pe.maximize)
# Constraints
m.con = pe.Constraint(expr = 3*m.x1 + 2*m.x2 <= 18)
# Solve problem using GLPK solver
glpk.solve(m).write()
# Print results
print('x1 =',m.x1.value)
print('x2 =',m.x2.value)
print('Optimal value =',m.obj())
# Explanation: This section demonstrates how to solve the optimization problem using Pyomo and GLPK.

## GUROBIPY ([link](https://pypi.org/project/gurobipy/))

Gurobi is a powerful commercial solver known for its performance. It is widely used in industry for large-scale optimization problems.

In [ ]:
# Model
m = gp.Model()
# Variables
x1 = m.addVar(vtype=GRB.CONTINUOUS,lb=0,ub=4)
x2 = m.addVar(vtype=GRB.CONTINUOUS,lb=0,ub=6)
# Objective function
m.setObjective(3*x1 + 5*x2, GRB.MAXIMIZE)
# Constraints
m.addConstr(3*x1 + 2*x2 <= 18)
# Solve problem using GLPK solver
m.optimize()
# Print results
print('x1 =',x1.X)
print('x2 =',x2.X)
print('Optimal value =',m.ObjVal)
# Explanation: This section demonstrates how to solve the optimization problem using Gurobi.